# Run a test suite

In [18]:
import os, pprint

In [19]:
EXAMPLES_FOLDER = "../examples"

In [20]:
[f for f in os.listdir(EXAMPLES_FOLDER) if f not in [".DS_Store", ".ipynb_checkpoints"]]

['.tools',
 'change_case',
 'multivariate',
 'results',
 'sacurine',
 'workflow-test-suite-full.yml',
 'workflow-test-suite-min.yml',
 'workflows.json']

In [21]:
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-min.yml")

In [22]:
import ruamel.yaml as yaml
with open(suite_conf_filename, "r") as fp:
    data = yaml.load(fp)
    print yaml.round_trip_dump(data, indent=2, block_seq_indent=3)

enable_logger: false
workflows:
  change_case:
    expected:
      OutputText: change_case/expected_output
    inputs:
      InputText: change_case/input
    file: change_case/workflow.ga
  multivariate:
    expected:
      variableMetadata_out: multivariate/variableMetadata_out
      sampleMetadata_out: multivariate/sampleMetadata_out
    inputs:
      DataMatrix: multivariate/dataMatrix.tsv
      SampleMetadata: multivariate/sampleMetadata.tsv
      VariableMetadata: multivariate/variableMetadata.tsv
    params:
      3:
        predI: '1'
        respC: gender
        orthoI: NA
        testL: 'FALSE'
    file: multivariate/workflow.ga



In [23]:
from wft4galaxy import WorkflowTestConfiguration

In [24]:
help(WorkflowTestConfiguration.load)

Help on function load in module wft4galaxy:

load(filename='workflows.yml', workflow_test_name=None, file_format='YAML', output_folder=None)
    Load the configuration of a workflow test suite or a single workflow test
    from a YAML or JSON configuration file.
    
    :type filename: str
    :param filename: the path of the file containing the suite definition
    
    :type workflow_test_name: str
    :param workflow_test_name: the optional name of a workflow test
    
    :type file_format: str
    :param file_format: ``YAML`` or ``JSON``
    
    :rtype: dict or :class:`WorkflowTestConfiguration`
    :return: a dictionary containing the configuration of a workflow test suite or
        a single workflow test configuration (i.e., an instance of :class:`WorkflowTestConfiguration`)



In [25]:
suite_conf = WorkflowTestConfiguration.load(suite_conf_filename)

In [26]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(suite_conf)

{   'enable_logger': False,
    'galaxy_api_key': None,
    'galaxy_url': None,
    'output_folder': 'results',
    'workflows': {   'change_case': WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText],
                     'multivariate': WorkflowTestConfig: name=multivariate, file=multivariate/workflow.ga, inputs=[DataMatrix,SampleMetadata,VariableMetadata], expected_outputs=[variableMetadata_out,sampleMetadata_out]}}


In [27]:
import pprint
pprint.pprint(suite_conf["workflows"])

{'change_case': WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText],
 'multivariate': WorkflowTestConfig: name=multivariate, file=multivariate/workflow.ga, inputs=[DataMatrix,SampleMetadata,VariableMetadata], expected_outputs=[variableMetadata_out,sampleMetadata_out]}


In [28]:
print(suite_conf["workflows"]["change_case"])

WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText]


In [29]:
from wft4galaxy import WorkflowTestSuite
suite_runner = WorkflowTestSuite()

2016-11-09 07:18:11,054 DEBUG: Initializing the existing WorkflowLoader instance...


In [30]:
suite_runner.run_test_suite(suite_conf, enable_debug=True)

Workflow Test: 'change_case' ... 2016-11-09 07:18:15,734 DEBUG: Using default params
2016-11-09 07:18:15,736 DEBUG: Checking required tools ...
2016-11-09 07:18:15,861 DEBUG: Missing tools: None
2016-11-09 07:18:15,862 DEBUG: Checking required tools: DONE
2016-11-09 07:18:16,382 INFO: Create a history '_WorkflowTestHistory_4cc53eb5-a644-11e6-99d7-a8667f33b7fc' (id: u'bf0806da5b28c6d9')
2016-11-09 07:18:18,296 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: bf0806da5b28c6d9) running ...
2016-11-09 07:18:30,049 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: bf0806da5b28c6d9) executed
2016-11-09 07:18:30,052 INFO: Checking test output: ...
2016-11-09 07:18:30,053 DEBUG: Checking OUTPUT 'OutputText' ...
2016-11-09 07:18:30,139 DEBUG: Downloaded output OutputText: dataset_id '7e1ddb768ae0c642', filename 'results/change_case/OutputText'
2016-11-09 07:18:30,141 DEBUG: Configured comparator function: None
2016-11-09 07:18:30,144 DEBUG: Using default com

In [31]:
test_results = suite_runner.run_tests(suite_conf, enable_debug=True)

2016-11-09 07:20:32,720 DEBUG: Using default params
2016-11-09 07:20:32,720 DEBUG: Checking required tools ...
2016-11-09 07:20:32,852 DEBUG: Missing tools: None
2016-11-09 07:20:32,853 DEBUG: Checking required tools: DONE
2016-11-09 07:20:35,774 INFO: Create a history '_WorkflowTestHistory_9e6b786e-a644-11e6-9512-a8667f33b7fc' (id: u'81991b32733a4fc4')
2016-11-09 07:20:38,189 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: 81991b32733a4fc4) running ...
2016-11-09 07:20:51,383 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: 81991b32733a4fc4) executed
2016-11-09 07:20:51,384 INFO: Checking test output: ...
2016-11-09 07:20:51,385 DEBUG: Checking OUTPUT 'OutputText' ...
2016-11-09 07:20:51,554 DEBUG: Downloaded output OutputText: dataset_id 'a8539f6d9115ffe7', filename 'results/change_case/OutputText'
2016-11-09 07:20:51,555 DEBUG: Configured comparator function: None
2016-11-09 07:20:51,556 DEBUG: Using default comparator....
2016-11-09 07:20:51,5

In [32]:
print test_results

[Test 9e6b786e-a644-11e6-9512-a8667f33b7fc: workflow _WorkflowTest_Change Case (imported from API), intputs=[InputText], outputs=[OutputText: OK], Test ab290711-a644-11e6-a351-a8667f33b7fc: workflow _WorkflowTest_Multivariate (imported from API), intputs=[DataMatrix,SampleMetadata,VariableMetadata], outputs=[variableMetadata_out: OK, sampleMetadata_out: OK]]


In [33]:
for r in test_results:
    print("Test %s:\n\t - workflow: [%s] \n\t - results: %r" % (r.test_id, r.workflow.name, r.results))

Test 9e6b786e-a644-11e6-9512-a8667f33b7fc:
	 - workflow: [_WorkflowTest_Change Case (imported from API)] 
	 - results: {u'OutputText': True}
Test ab290711-a644-11e6-a351-a8667f33b7fc:
	 - workflow: [_WorkflowTest_Multivariate (imported from API)] 
	 - results: {u'variableMetadata_out': True, u'sampleMetadata_out': True}


In [34]:
a_result = test_results[0]

In [35]:
help(a_result)

Help on instance of WorkflowTestResult in module wft4galaxy:

class WorkflowTestResult
 |  Class for representing the result of a workflow test.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, test_id, workflow, inputs, outputs, output_history, expected_outputs, missing_tools, results, output_file_map, output_folder='results')
 |  
 |  __repr__(self)
 |  
 |  __str__(self)
 |  
 |  check_output(self, output)
 |      Assert whether the actual `output` is equal to the expected accordingly
 |      to its associated `comparator` function.
 |      
 |      :type output: str or dict
 |      :param output: output name
 |      
 |      :rtype: bool
 |      :return: ``True`` if the test is passed; ``False`` otherwise.
 |  
 |  check_outputs(self)
 |      Return a map of pairs <OUTPUT_NAME>:<RESULT>, where <RESULT> is ``True``
 |      if the actual `OUTPUT_NAME` is equal to the expected accordingly
 |      to its associated `comparator` function.
 |      
 |      :rtype: dict
 |      :ret

In [36]:
print "Outputs: ", a_result.results.keys()

Outputs:  [u'OutputText']


In [37]:
a_result.passed(), a_result.failed()

(True, False)

In [39]:
a_result.check_output("OutputText")

True